In [45]:
# %load 200_tweets_to_parquet.py
#!/usr/bin/env python3
"""
Created on Mon Sep 23 13:52:14 2019

@author: hduser
"""
import time
import arrow
today_dt = arrow.now().format('YYYYMMDD')

import tarfile
import json
import os

# Comment out code that doesnt need to run every time
#tf = tarfile.open("geotagged_tweets_20160812-0912.tar.gz")
#tf.extractall()

#os.system("shuf -n 1000 geotagged_tweets_20160812-0912.jsons > geotagged_tweets_20160812-0912.1000.jsons")
#os.system("shuf -n 5000 geotagged_tweets_20160812-0912.jsons > geotagged_tweets_20160812-0912.5000.jsons")
#os.system("shuf -n 10000 geotagged_tweets_20160812-0912.jsons > geotagged_tweets_20160812-0912.10000.jsons")

tweets_data = []
tweets_file = open('tweets.' + today_dt + '.json', "r")
print(tweets_file)
from pandas.io.json import json_normalize

counter = 0
cols = ['created_at','place.country','place.country_code','place.name','is_quote_status','lang','source','text','user.created_at','user.description','user.name','user.location','place.place_type','user.screen_name','in_reply_to_screen_name']



<open file u'tweets.20200329.json', mode 'r' at 0x7fdc4b5dae40>


In [46]:
for line in tweets_file:
    counter = counter + 1
    if counter % 10000 == 0:
        print(counter)
        tweets_df = json_normalize(tweets_data)
        neat_df = tweets_df[cols]
        neat_df.to_parquet("tweets." + today_dt + ".{}.parquet".format(counter), compression='GZIP')
        tweets_data = []
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue

tweets_df = json_normalize(tweets_data)
neat_df = tweets_df[cols]
neat_df.to_parquet("tweets." + today_dt + ".{}.parquet".format(counter),compression='GZIP')


10000
20000
30000
40000
50000
60000


,contributors,coordinates,coordinates.coordinates,coordinates.type,created_at,display_text_range,entities.hashtags,entities.media,entities.symbols,entities.urls,...,user.profile_use_background_image,user.protected,user.screen_name,user.statuses_count,user.time_zone,user.translator_type,user.url,user.utc_offset,user.verified,withheld_in_countries
0,NaN,NaN,NaN,NaN,Sun Mar 29 15:31:39 +0000 2020,NaN,"[{u'indices': [72, 84], u'text': u'coronavírus'}]",NaN,[],[],...,True,False,CoronaBotBR,30078.0,NaN,none,None,NaN,False,NaN
1,NaN,NaN,NaN,NaN,Sun Mar 29 15:31:39 +0000 2020,NaN,[],NaN,[],[],...,True,False,morgandawn6,62632.0,NaN,none,None,NaN,False,NaN
2,NaN,NaN,NaN,NaN,Sun Mar 29 15:31:39 +0000 2020,NaN,[],NaN,[],"[{u'url': u'https://t.co/sNtPf3cvXE', u'indice...",...,True,False,yuggy52,86604.0,NaN,none,None,NaN,False,NaN
3,NaN,NaN,NaN,NaN,Sun Mar 29 15:31:39 +0000 2020,NaN,[],[{u'additional_media_info': {u'monetizable': F...,[],[],...,True,False,FBiometric,676.0,NaN,none,https://medrecords.emrflobamer.net,NaN,False,NaN
4,NaN,NaN,NaN,NaN,Sun Mar 29 15:31:39 +0000 2020,NaN,"[{u'indices': [90, 98], u'text': u'COVID19'}]",NaN,[],[],...,True,False,downtownnigeria,353.0,NaN,none,None,NaN,False,NaN


In [47]:
# This creates a python list of strings with json data in the string.  there's a lot of fields, and I can either
# cherry pick the fields I want, or figure out how to read them all in and subset them once I have them in a nice dataframe

import datetime
import glob
import os
from datetime import datetime  
from datetime import timedelta 


print("Start of program")

path = ''                     # use your path
all_files = glob.glob(os.path.join(path, "tweets." + today_dt + ".*.parquet"))     # advisable to use os.path.join as this makes concatenation 

all_files.sort()
print(all_files)

Start of program
[u'tweets.20200329.10000.parquet', u'tweets.20200329.12340.parquet', u'tweets.20200329.20000.parquet', u'tweets.20200329.30000.parquet', u'tweets.20200329.30750.parquet', u'tweets.20200329.40000.parquet', u'tweets.20200329.50000.parquet', u'tweets.20200329.60000.parquet', u'tweets.20200329.64256.parquet']


In [49]:
import pandas as pd

for i,f in enumerate(all_files):
    parquet_chunk = pd.read_parquet(f)
    print("file read " + f)
    print(parquet_chunk.head())
    if i == 0:
        print("first iter... initialize dataframe with all the data in this chunk")
        tweets = parquet_chunk
    else:
        tweets = tweets.append(parquet_chunk, ignore_index=True)

tweets.to_parquet("tweets_all." + today_dt + ".parquet",compression='gzip')


file read tweets.20200329.10000.parquet
                       created_at place.country place.country_code place.name  \
0  Sun Mar 29 14:49:56 +0000 2020          None               None       None   
1  Sun Mar 29 14:49:56 +0000 2020          None               None       None   
2  Sun Mar 29 14:49:56 +0000 2020          None               None       None   
3  Sun Mar 29 14:49:56 +0000 2020          None               None       None   
4  Sun Mar 29 14:49:56 +0000 2020          None               None       None   

   is_quote_status lang                                             source  \
0              0.0   es  <a href="http://twitter.com/download/iphone" r...   
1              1.0   en  <a href="https://mobile.twitter.com" rel="nofo...   
2              0.0   es  <a href="http://twitter.com/download/android" ...   
3              0.0   in  <a href="http://twitter.com/download/iphone" r...   
4              0.0   es  <a href="http://twitter.com/download/iphone" r...   

    

file read tweets.20200329.40000.parquet
                       created_at place.country place.country_code place.name  \
0  Sun Mar 29 15:13:33 +0000 2020          None               None       None   
1  Sun Mar 29 15:13:33 +0000 2020          None               None       None   
2  Sun Mar 29 15:13:33 +0000 2020          None               None       None   
3  Sun Mar 29 15:13:33 +0000 2020          None               None       None   
4  Sun Mar 29 15:13:33 +0000 2020          None               None       None   

   is_quote_status lang                                             source  \
0              0.0   th  <a href="http://twitter.com/download/iphone" r...   
1              0.0  und  <a href="http://twitter.com/download/iphone" r...   
2              0.0   es  <a href="http://twitter.com/download/android" ...   
3              0.0   fr  <a href="https://dlvrit.com/" rel="nofollow">d...   
4              0.0   pt  <a href="https://mobile.twitter.com" rel="nofo...   

    